<a href="https://colab.research.google.com/github/EvagAIML/014-NLP-Model-v1/blob/main/Medical_Diagnostic_Tool_v3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Medical Diagnostic Assistant: AI-Powered Question Answering

## Problem Statement

### Business Context
The healthcare industry is rapidly evolving, with professionals facing increasing challenges in managing vast volumes of medical data while delivering accurate and timely diagnoses. The need for quick access to comprehensive, reliable, and up-to-date medical knowledge is critical for improving patient outcomes and ensuring informed decision-making in a fast-paced environment.

Healthcare professionals often encounter information overload, struggling to sift through extensive research and data to create accurate diagnoses and treatment plans. This challenge is amplified by the need for efficiency, particularly in emergencies, where time-sensitive decisions are vital. Furthermore, access to trusted, current medical information from renowned manuals and research papers is essential for maintaining high standards of care.

### Objective
As an AI specialist, the task is to develop a **Retrieval-Augmented Generation (RAG)** solution using renowned medical manuals to address healthcare challenges. The objective is to **understand** issues like information overload, **apply** AI techniques to streamline decision-making, **analyze** its impact on diagnostics and patient outcomes, **evaluate** its potential to standardize care practices, and **create** a functional prototype demonstrating its feasibility and effectiveness.

**Common Questions to Answer**

1. **Diagnostic Assistance**: "What are the common symptoms and treatments for pulmonary embolism?"
2. **Drug Information**: "Can you provide the trade names of medications used for treating hypertension?"
3. **Treatment Plans**: "What are the first-line options and alternatives for managing rheumatoid arthritis?"
4. **Specialty Knowledge**: "What are the diagnostic steps for suspected endocrine disorders?"
5. **Critical Care Protocols**: "What is the protocol for managing sepsis in a critical care unit?"

### Data Description
The **Merck Manuals** are medical references published by the American pharmaceutical company Merck & Co., that cover a wide range of medical topics, including disorders, tests, diagnoses, and drugs.

The manual is provided as a PDF: `014-NLP-PROJ-medical_diagnosis_manual_19.pdf`

## Part 1: Setup and Initialization
Installing necessary libraries and setting up the environment.

In [1]:
# Installation for GPU llama-cpp-python
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install llama-cpp-python==0.2.28 --force-reinstall --no-cache-dir -q

# Installing other dependencies
!pip install huggingface_hub==0.35.3 pandas==2.2.2 tiktoken==0.12.0 pymupdf==1.26.5 langchain==0.3.27 langchain-community==0.3.31 chromadb==1.1.1 sentence-transformers==5.1.1 numpy==2.3.3 -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 53.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 330.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 312.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.6/16.6 MB 357.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 294.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-adk 1.21.0 requires opentelemetry-api<=1.37.0,>=1.37.0, but you have opentelemetry-api 1.39.1 which is incompatible.
google-adk 1.21.0 requires opentelemetry-sdk<=1.37.0,>=1.37.0, but you have opentelemetry-sdk 1.39.1 which is incompatible.

**Note:** Please restart the runtime after installation.

In [2]:
import os
import pandas as pd
from huggingface_hub import hf_hub_download
from llama_cpp import Llama
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.embeddings.sentence_transformer import SentenceTransformerEmbeddings
from langchain_community.vectorstores import Chroma

# Define the questions globally
questions = [
    "What are the common symptoms and treatments for pulmonary embolism?",
    "Can you provide the trade names of medications used for treating hypertension?",
    "What are the first-line options and alternatives for managing rheumatoid arthritis?",
    "What are the diagnostic steps for suspected endocrine disorders?",
    "What is the protocol for managing sepsis in a critical care unit?"
]

## Part 2: Question Answering using LLM (Vanilla)
Here we load the LLM and ask the 5 questions without any external context to establish a baseline.

In [4]:
model_name_or_path = "TheBloke/Mistral-7B-Instruct-v0.2-GGUF"
model_basename = "mistral-7b-instruct-v0.2.Q4_K_M.gguf"

model_path = hf_hub_download(
    repo_id=model_name_or_path,
    filename=model_basename
)

# Initialize LLM
llm = Llama(
    model_path=model_path,
    n_ctx=4096,        # Context window
    n_gpu_layers=-1,   # Offload all to GPU suitable for T4
    n_batch=512,
    verbose=True
)

AVX = 1 | AVX_VNNI = 0 | AVX2 = 1 | AVX512 = 1 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 1 | SSE3 = 1 | SSSE3 = 1 | VSX = 0 | 


In [6]:
def generate_response(query, max_tokens=256, temperature=0.1, top_p=0.95, top_k=50, system_prompt=None):
    if system_prompt:
        prompt = f"[INST] {system_prompt}\n{query} [/INST]"
    else:
        prompt = f"[INST] {query} [/INST]"

    model_output = llm(
        prompt=prompt,
        max_tokens=max_tokens,
        temperature=temperature,
        top_p=top_p,
        top_k=top_k
    )
    return model_output['choices'][0]['text'].strip()

### Baseline Answers (Vanilla)

In [7]:
vanilla_results = []
for i, q in enumerate(questions, 1):
    print(f"--- Question {i} ---")
    print(f"Q: {q}")
    ans = generate_response(q)
    print(f"A: {ans}\n")
    vanilla_results.append(ans)

--- Question 1 ---
Q: What are the common symptoms and treatments for pulmonary embolism?
A: A pulmonary embolism (PE) is a serious condition that occurs when a blood clot or other foreign substance travels to the lungs and blocks one or more of the arteries supplying the lungs with blood. This can lead to decreased oxygen supply to the body, which can be life-threatening if not treated promptly. Here are some common symptoms of pulmonary embolism:

1. Shortness of breath: This is often the first and most prominent symptom of PE. The person may feel like they're unable to get enough air or that they're breathing too hard for the level of activity they're engaged in.
2. Chest pain: PE can cause sharp, stabbing chest pain that worsens with deep breaths or physical activity. The pain may be described as feeling like a heavy weight on the chest.
3. Rapid heart rate: PE can cause the heart to beat faster than normal in an attempt to pump more blood through the body and deliver oxygen to the

Llama.generate: prefix-match hit


A: Yes, I can provide a list of some trade names of medications commonly used to treat hypertension (high blood pressure). Please note that this is not an exhaustive list and there are many other brand names available depending on the specific medication and region. Here are some examples:

1. ACE inhibitors: Lisinopril (Prinivil, Zestril), Enalapril (Vasotec), Quinapril (Accupril)
2. Angiotensin II receptor blockers (ARBs): Losartan (Cozaar), Valsartan (Diovan), Candesartan (Atacand)
3. Calcium channel blockers: Amlodipine (Norvasc), Diltiazem (Cardizem, Tiazac), Nifedipine (Procardia, Adalat)
4. Diuretics: Furosemide (Lasix), Hydrochlorothiazide (Microzide, Esidrix), Spironolactone (Aldactone)
5. Beta blockers: Metoprolol (Lopressor, Toprol XL), Propr

--- Question 3 ---
Q: What are the first-line options and alternatives for managing rheumatoid arthritis?


Llama.generate: prefix-match hit


A: The first-line options for managing rheumatoid arthritis (RA) typically involve a combination of medications, lifestyle modifications, and physical therapy to help reduce inflammation, alleviate symptoms, and prevent joint damage. Here are some common first-line treatments:

1. Disease-modifying antirheumatic drugs (DMARDs): These medications work by slowing down or stopping the progression of RA. Commonly used DMARDs include methotrexate, sulfasalazine, hydroxychloroquine, leflunomide, and azathioprine.
2. Biologic agents: For patients with severe or refractory RA who do not respond to DMARDs, biologic agents may be considered as a first-line option. These medications target specific molecules involved in the inflammatory process and include tumor necrosis factor (TNF) inhibitors (e.g., adalimumab, etanercept, infliximab), interleukin (IL)-1 inhibitors (e.g., anakinra), and IL-6 receptor inhibitors (e.g.,

--- Question 4 ---
Q: What are the diagnostic steps for suspected endocrine 

Llama.generate: prefix-match hit


A: Diagnosing endocrine disorders involves a systematic approach that includes a thorough medical history, physical examination, and various diagnostic tests. Here are some general diagnostic steps for suspected endocrine disorders:

1. Medical History: Obtaining a detailed medical history is essential to understand the patient's symptoms, duration, and any associated conditions. This may include questions about family history, lifestyle factors, medications, and past surgical procedures.
2. Physical Examination: A physical examination can help identify signs of endocrine disorders, such as changes in growth patterns, skin texture, body hair distribution, or organ enlargement.
3. Laboratory Tests: Various laboratory tests can be used to diagnose endocrine disorders. These may include measurements of hormone levels, blood glucose levels, and other biochemical markers. Some common tests for endocrine disorders include:
   - Thyroid function tests (TSH, T3, T4)
   - Glucose tolerance test

Llama.generate: prefix-match hit


A: Sepsis is a life-threatening condition caused by the body's response to an infection. In a critical care unit, managing sepsis requires a multidisciplinary approach and prompt recognition and intervention. Here are some key steps in the protocol for managing sepsis in a critical care unit:

1. Early Recognition: Identify patients at risk of developing sepsis based on clinical signs and laboratory results. The early warning scores, such as Quick Sequential Organ Failure Assessment (qSOFA) or Systemic Inflammatory Response Syndrome (SIRS), can help identify patients who may be at risk.
2. Resuscitation: Provide fluid resuscitation to maintain adequate tissue perfusion and organ function. The goal is to achieve a mean arterial pressure of 65 mmHg or higher and a central venous oxygen saturation (ScvO2) of greater than 70%.
3. Antibiotics: Administer broad-spectrum antibiotics as soon as possible based on the suspected infection site and microbiological cultures if available.
4. Source 

### Observations (Vanilla LLM)
- **General Knowledge:** The model answers general medical questions reasonably well (e.g., symptoms of PE) based on its training data.
- **Drug Names:** It may struggle with specific lists of trade names versus generic names without a lookup source.
- **Hallucination Risk:** Without RAG, there is no guarantee that the "protocols" or "steps" mentioned are from the specific Merck Manual standard or guidelines.

## Part 3: LLM with Prompt Engineering & Parameter Tuning
We will test 5 different combinations of prompts and hyperparameters to observe how the style and quality of answers change.

In [8]:
# Define 5 Configurations
configs = [
    {"name": "High Creativity", "temp": 0.9, "sys": "You are a helpful medical assistant.", "top_k": 100},
    {"name": "Strict Professional", "temp": 0.0, "sys": "You are a concise, professional doctor. Answer only what is asked.", "top_k": 10},
    {"name": "Chain of Thought", "temp": 0.3, "sys": "Think step-by-step. Explain the reasoning before giving the final answer.", "top_k": 40},
    {"name": "ELI5", "temp": 0.7, "sys": "Explain like I am 5 years old.", "top_k": 50},
    {"name": "Balanced", "temp": 0.4, "sys": "You are a knowledgeable assistant. Provide detailed clinical information.", "top_k": 50}
]

# We will test these on Question 5 (Sepsis Protocol) as a case study
test_q = questions[4]
print(f"Testing Configuration on Question: {test_q}\n")

for cfg in configs:
    print(f"--- {cfg['name']} ---")
    ans = generate_response(
        test_q,
        temperature=cfg['temp'],
        top_k=cfg['top_k'],
        system_prompt=cfg['sys']
    )
    print(f"{ans}\n")

Testing Configuration on Question: What is the protocol for managing sepsis in a critical care unit?

--- High Creativity ---


Llama.generate: prefix-match hit


Sepsis is a life-threatening condition that can rapidly deteriorate a patient's health. In a critical care unit, managing sepsis requires a multidisciplinary approach and prompt intervention. Here are the general steps for managing sepsis in a critical care unit:

1. Recognize and diagnose sepsis: Early recognition and diagnosis of sepsis are crucial for effective treatment. Look for signs and symptoms such as fever or hypothermia, tachycardia or bradycardia, respiratory distress, altered mental status, and lactic acidosis. Laboratory tests, including complete blood count, lactate level, and blood cultures, can help confirm the diagnosis.
2. Provide oxygenation and ventilation support: Hypoxemia is a common complication of sepsis. Monitor arterial oxygen saturation and provide oxygen therapy as needed. Intubation and mechanical ventilation may be required in severe cases.
3. Fluid resuscitation: Sepsis can cause fluid shifts, leading to hypovolemia. Provide intravenous fluids to mainta

Llama.generate: prefix-match hit


The protocol for managing sepsis in a critical care unit involves the following steps:

1. Recognition and early identification of sepsis based on clinical suspicion and use of validated scoring systems like Sequential Organ Failure Assessment (SOFA) or Quick Sequential Organ Failure Assessment (qSOFA).
2. Immediate initiation of appropriate antibiotic therapy based on culture results, if available, or suspected source of infection.
3. Fluid resuscitation to maintain adequate tissue perfusion and organ function, with careful monitoring of fluid balance and hemodynamic status.
4. Administration of vasopressors and/or inotropes as needed to maintain mean arterial pressure (MAP) ≥65 mmHg.
5. Close monitoring of vital signs, laboratory values, and organ function, including lactate levels, urine output, and oxygenation status.
6. Adjustment of antibiotic therapy based on culture results and susceptibility testing.
7. Supportive care measures such as mechanical ventilation, renal replacement

...

Sepsis is a life-threatening condition caused by the body's response to an infection, and managing it effectively in a critical care unit requires a systematic approach. Here are the key steps in the sepsis management protocol:

1. Early recognition: Recognizing sepsis early is crucial for effective treatment. Look for signs of infection, such as fever or hypothermia, tachycardia or bradycardia, altered mental status, and lactic acidosis. Use the Sequential Organ Failure Assessment (SOFA) score or Quick Sequential Organ Failure Assessment (qSOFA) to identify patients at risk of sepsis.

2. Initial assessment: Once sepsis is suspected, perform a thorough assessment, including vital signs, oxygen saturation, and urine output. Consider obtaining laboratory tests such as complete blood count, lactate level, and cultures from potential infection sites.

3. Fluid resuscitation: Aggressively administer fluids to maintain adequate tissue perfusion and organ function. Use crystalloids initially

Llama.generate: prefix-match hit


Sepsis is when your body gets really sick from an infection, and we need to help it get better in a special hospital called a critical care unit. Here's what the doctors and nurses do:

1. Give medicines to make the body stronger and fight the infection.
2. Give fluids through an IV to keep the body hydrated.
3. Monitor the heart rate, breathing, temperature, and blood pressure closely.
4. Provide oxygen if needed through a mask or a tube in the nose.
5. Help the body remove waste by inserting a catheter in the bladder.
6. Keep the patient comfortable by providing good bed rest and pain relief.
7. Give antibiotics to kill the bacteria causing the infection.
8. Provide nutrients through a feeding tube if the patient can't eat or drink on their own.
9. Collaborate with other specialists as needed, like kidney doctors or surgeons.
10. Support the family and keep them informed throughout the treatment process.

--- Balanced ---


Llama.generate: prefix-match hit


Sepsis is a life-threatening condition caused by the body's response to an infection, which can lead to tissue damage and organ failure. Managing sepsis in a critical care unit requires a multidisciplinary approach and prompt recognition and intervention. Here are the key steps in managing sepsis in a critical care unit:

1. Early Recognition: The first step is early recognition of sepsis based on clinical suspicion, vital signs, laboratory findings, and patient history. The Sequential [Sepsis-related] Organ Failure Assessment (SOFA) score can be used to identify patients at risk for developing sepsis or worsening sepsis.
2. Fluid Resuscitation: Aggressive fluid resuscitation is a cornerstone of sepsis management in the critical care unit. The goal is to maintain adequate tissue perfusion and end-organ function. Crystalloid solutions are typically used for initial resuscitation, with colloids or blood products considered if adequate volume resuscitation is not achieved.
3. Antimicrobia

### Observations (Prompt Engineering)
- **Temperature:** Low temperature (0.0) yields very deterministic and often drier responses, ideal for protocols.
- **Persona:** The "ELI5" prompt successfully simplifies the complex medical jargon.
- **Chain of Thought:** Encourages the model to break down the sepsis protocol into logical steps (Identify -> Resuscitate -> Antibiotics).

## Part 4: Data Preparation for RAG
Ingesting the Merck Manual PDF, chunking it, embedding it, and storing it in a Vector Database.

In [29]:
doc_path = "014-NLP-PROJ-medical_diagnosis_manual_19.pdf"

if os.path.exists(doc_path):
    loader = PyMuPDFLoader(doc_path)
    docs = loader.load()
    print(f"Loaded {len(docs)} pages.")
else:
    print(f"File {doc_path} not found. Please ensure it is in the working directory.")

Loaded 4114 pages.


In [30]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200
)
chunks = text_splitter.split_documents(docs)
print(f"Total Chunks created: {len(chunks)}")

Total Chunks created: 18020


In [31]:
import os

file_path = '014-NLP-PROJ-medical_diagnosis_manual_19.pdf'
if os.path.exists(file_path):
    print(f"The file '{file_path}' exists in the current working directory.")
else:
    print(f"The file '{file_path}' DOES NOT exist in the current working directory. Please upload it.")

The file '014-NLP-PROJ-medical_diagnosis_manual_19.pdf' exists in the current working directory.


In [32]:
embedding_model = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

vector_db = Chroma.from_documents(
    documents=chunks,
    embedding=embedding_model,
    persist_directory="./chroma_db_medical_final"
)

# Default Retriever
retriever = vector_db.as_retriever(search_kwargs={"k": 3})

## Part 5: Question Answering using RAG
We implement the RAG pipeline and fine-tune parameters to get the best evidence-based answers.

In [33]:
def generate_rag_answer(query, k=3, temperature=0.1, max_tokens=512):
    # Retrieve
    retrieved_docs = vector_db.similarity_search(query, k=k)
    context = "\n\n".join([d.page_content for d in retrieved_docs])

    # Construct Prompt
    sys_msg = "You are a medical assistant. Use the following Context to answer the Question. If the answer is not in the context, say so."
    user_msg = f"Context:\n{context}\n\nQuestion: {query}"

    prompt = f"[INST] {sys_msg}\n{user_msg} [/INST]"

    # Generate
    output = llm(
        prompt=prompt,
        max_tokens=max_tokens,
        temperature=temperature,
        stop=["[/INST]"]
    )
    return output['choices'][0]['text'].strip(), context

In [34]:
# RAG Parameter Tuning (5 Combinations)
rag_configs = [
    {"name": "C1: Low Context, Precise", "k": 2, "temp": 0.0},
    {"name": "C2: Standard", "k": 3, "temp": 0.1},
    {"name": "C3: High Context", "k": 5, "temp": 0.1},
    {"name": "C4: Creative", "k": 3, "temp": 0.7},
    {"name": "C5: Max Context, Strict", "k": 7, "temp": 0.0}
]

print("Running RAG Tuning on Question: '...rheumatoid arthritis?'")
tune_q = questions[2]

for cfg in rag_configs:
    print(f"--- {cfg['name']} ---")
    ans, _ = generate_rag_answer(tune_q, k=cfg['k'], temperature=cfg['temp'])
    print(f"A: {ans[:300]}...\n")

Running RAG Tuning on Question: '...rheumatoid arthritis?'
--- C1: Low Context, Precise ---


Llama.generate: prefix-match hit


A: The first-line options for managing rheumatoid arthritis (RA) include:

1. Treating the underlying disorder with immunosuppression or antibiotics, as determined by the diagnosis.
2. Symptomatic treatment of joint inflammation with Nonsteroidal Anti-Inflammatory Drugs (NSAIDs).
3. Pain without inflam...

--- C2: Standard ---


Llama.generate: prefix-match hit


A: The first-line options for managing rheumatoid arthritis (RA) include:

1. Disease-modifying antirheumatic drugs (DMARDs): These medications are used early in the treatment of RA to reduce inflammation and prevent erosions and progressive deformity. They are often used in combination.
2. Nonsteroida...

--- C3: High Context ---


...

A: The first-line options for managing rheumatoid arthritis (RA) involve a combination of medications known as disease-modifying antirheumatic drugs (DMARDs). These drugs help control symptoms and slow down the progression of the disease. Some common DMARDs include methotrexate, sulfasalazine, leflunom...

--- C4: Creative ---


Llama.generate: prefix-match hit


A: The first-line options for managing rheumatoid arthritis (RA) involve treating the underlying disorder with immunosuppression or antibiotics if there is a systemic infection. For joint inflammation, nonsteroidal anti-inflammatory drugs (NSAIDs) are usually prescribed symptomatically. In cases of pai...

--- C5: Max Context, Strict ---


Llama.generate: prefix-match hit


A: The first-line options for managing rheumatoid arthritis (RA) include supportive measures such as rest, nutrition, physical measures, and analgesics like acetaminophen or nonsteroidal anti-inflammatory drugs (NSAIDs). For inflammation reduction and disease progression control, disease-modifying anti...



### Final RAG Answers (Best Configuration: k=3, temp=0.1)

In [35]:
rag_responses = []
rag_contexts = []

for i, q in enumerate(questions, 1):
    print(f"--- RAG Question {i} ---")
    print(f"Q: {q}")
    ans, ctx = generate_rag_answer(q, k=3, temperature=0.1)
    print(f"A: {ans}\n")
    rag_responses.append(ans)
    rag_contexts.append(ctx)

--- RAG Question 1 ---
Q: What are the common symptoms and treatments for pulmonary embolism?


Llama.generate: prefix-match hit


A: The common symptoms for pulmonary embolism include dyspnea (shortness of breath), pleuritic chest pain, cough, tachypnea (rapid breathing), tachycardia (fast heart rate), hypotension (low blood pressure), and a loud pulmonic component of the 2nd heart sound.

The treatment for pulmonary embolism typically involves anticoagulants to prevent clot formation and growth. In some cases, clot dissolution with thrombolytics or surgical removal may be necessary. Preventive measures include anticoagulants and the insertion of an inferior vena caval filter. It's important to note that the treatment for pulmonary embolism can vary depending on the severity and individual case.

--- RAG Question 2 ---
Q: Can you provide the trade names of medications used for treating hypertension?


Llama.generate: prefix-match hit


A: The context does not mention specific trade names of medications used for treating hypertension. It only mentions that ACE inhibitors and angiotensin II receptor blockers are drugs of choice for many patients with arterial hypertension, and diuretics are also used as part of the treatment regimen. However, it does not provide any specific trade names for these medications.

--- RAG Question 3 ---
Q: What are the first-line options and alternatives for managing rheumatoid arthritis?


...

A: The first-line options for managing rheumatoid arthritis (RA) include:

1. Disease-modifying antirheumatic drugs (DMARDs): These medications are used early and often in combination to reduce inflammation as a means of preventing erosions and progressive deformity.
2. Nonsteroidal anti-inflammatory drugs (NSAIDs): These medications are used to relieve joint inflammation and pain.
3. Acetaminophen: This medication is used for managing pain without inflammation.
4. Joint immobilization with a splint or sling: This can sometimes relieve pain.
5. Heat therapy: This may relieve muscle spasm around joints.
6. Cold therapy: This may be analgesic in inflammatory joint diseases.
7. Continued physical activity: Encouraged for cases of chronic arthritis.

Alternative options for managing RA include:

1. Immunosuppression: Used to treat systemic diseases that require suppression of the immune system.
2. Antibiotics: Used to treat infections that may be contributing to RA or complicating its mana

Llama.generate: prefix-match hit


A: The diagnostic steps for suspected endocrine disorders include:

1. Clinical evaluation: Recognizing symptoms that may indicate an endocrine disorder, although they can begin insidiously and be nonspecific, leading to a delayed diagnosis.
2. Laboratory testing: Measuring levels of the peripheral endocrine hormone, pituitary hormone, or both in the blood is essential for biochemical diagnosis due to the often subtle and nonspecific symptoms.
3. Detecting autoantibodies to each affected glandular tissue can help differentiate various causes of endocrine deficiencies, such as primary dysfunction (PDS) from other causes.
4. Elevated levels of pituitary tropic hormones suggest the hypothalamic-pituitary axis is intact, although some patients with type II PDS may have hypothalamic-pituitary insufficiency.
5. Lifelong follow-up is necessary due to the potential for decades passing before the appearance of all manifestations and the risk of unrecognized hypoparathyroidism or adrenal insuffi

Llama.generate: prefix-match hit


A: The protocol for managing sepsis in a critical care unit includes providing first aid by keeping the patient warm, controlling hemorrhage if present, checking and securing the airway and ventilation, turning the patient's head to one side if emesis occurs, and providing supplemental oxygen via face mask. If the shock is severe or if ventilation is inadequate, airway intubation with mechanical ventilation is necessary. Two large IV catheters should be inserted into separate peripheral veins, and a central venous line or intraosseous needle may be used as an alternative when prompt access to peripheral veins cannot be achieved. (Reference: Chapter 222, Approach to the Critically Ill Patient, Septic Shock section)



### Observations (RAG)
- **Specificity:** The answers are now directly referencing information found in the Merck Manual chunks.
- **Reduced Hallucination:** When asked about trade names (Q2), if the chunks do not contain them, the model is more likely to stick to what is present (generic names) or state limitations, rather than inventing names.
- **Context:** Increasing `k` helps when the answer is spread across multiple sections, but too high `k` can introduce noise.

## Part 6: Output Evaluation (LLM-as-a-Judge)
Evaluating the RAG responses for **Groundedness** and **Relevance**.

In [36]:
def evaluate_response(query, answer, context):
    # Groundedness Prompt
    ground_prompt = f"""
    [INST] You are a judge. Determine if the Answer is fully supported by the Context.
    Context: {context}
    Answer: {answer}
    Reply with 'YES' if supported, 'NO' if not.
    [/INST]
    """
    ground_score = llm(ground_prompt, max_tokens=10, temperature=0.0)['choices'][0]['text'].strip()

    # Relevance Prompt
    rel_prompt = f"""
    [INST] You are a judge. Determine if the Answer directly addresses the Question.
    Question: {query}
    Answer: {answer}
    Reply with 'YES' if relevant, 'NO' if not.
    [/INST]
    """
    rel_score = llm(rel_prompt, max_tokens=10, temperature=0.0)['choices'][0]['text'].strip()

    return ground_score, rel_score

print("--- Evaluation Results ---\n")
for i, (q, a, c) in enumerate(zip(questions, rag_responses, rag_contexts), 1):
    g, r = evaluate_response(q, a, c)
    print(f"Q{i}: Groundedness: {g} | Relevance: {r}")

--- Evaluation Results ---



Llama.generate: prefix-match hit
Llama.generate: prefix-match hit


Q1: Groundedness: YES, the answer is fully supported by the | Relevance: YES, the Answer directly addresses both the Question


Llama.generate: prefix-match hit
Llama.generate: prefix-match hit


Q2: Groundedness: NO. The context does not mention any specific trade | Relevance: NO, the answer does not directly address the question


...

Q3: Groundedness: YES, the answer is fully supported by the | Relevance: YES, the answer directly addresses the question by


Llama.generate: prefix-match hit
Llama.generate: prefix-match hit


Q4: Groundedness: YES, the answer is fully supported by the | Relevance: YES, the answer directly addresses the question by


Llama.generate: prefix-match hit
Llama.generate: prefix-match hit


Q5: Groundedness: YES. The answer is fully supported by the | Relevance: YES, the answer directly addresses the question by


## Part 7: Actionable Insights and Recommendations

### Key Takeaways
1. **RAG Efficiency:** The implementation of RAG significantly improves the reliability of the medical assistant by grounding answers in the trusted Merck Manual.
2. **Parameter Importance:** Tuning the chunk retrieval size (k) and temperature is crucial. `k=3` provided a good balance of context without overwhelming the model window.
3. **Prompt Engineering:** Specifying a persona (e.g., "Medical Assistant") helps maintain a professional tone suitable for healthcare applications.
4. **Evaluation:** The automated evaluation showed high relevance, validating the system's utility for diagnostic assistance.

### Business Recommendations
- **Deployment:** Deploy this prototype to a pilot group of clinicians to assist with information retrieval.
- **Data Expansion:** Integrate more specialized journals apart from the Merck Manual for wider coverage.
- **User Interface:** Build a frontend (Streamlit/React) to allow doctors to query easily from tablets.